In [6]:
import pandas as pd
import numpy as np
from IPython.display import display
from fastai.imports import *
from sklearn import metrics
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

hour_ahead = 24
n_estimators = 98
learning_rate = 0.1
Depth = 10
Min_leaf = 256

path = "/content/drive/MyDrive/Colab Notebooks/Project/star_data_set.csv"
Merge_all = pd.read_csv(path)
dti = pd.to_datetime(Merge_all['DateTime'], format='%Y-%m-%d %H', exact=False)
Merge_all.set_index(pd.Index(dti), inplace=True)
Merge_all.drop(['DateTime'], axis=1, inplace=True)

First = Merge_all.iloc[0:11000, :]
N_index = First.shape
Y_temp = First['FI']
Y = Y_temp.iloc[hour_ahead:N_index[0]]
Y = Y.to_numpy()
#Y.reset_index(drop=True, inplace=True)
#Y.columns = [''] * len(Y.columns)


X_temp1 = First['FI']
X_temp2 = First.loc[:, First.columns != 'FI']
#X_temp1 = First.drop(First.columns.difference(['FI', 'C_SE1 > FI', 'C_FI > SE1', 'C_SE3 > FI', 'C_FI > SE3', 'C_EE > FI', 'C_FI > EE', 'F_FI > SE1', 'F_SE1 > FI', 'F_FI > SE3', 'F_SE3 > FI', 'F_FI > EE', 'F_EE > FI', 'FI Buy', 'FI Sell']), axis=1)
#X_temp2 = First.drop(['FI', 'C_SE1 > FI', 'C_FI > SE1', 'C_SE3 > FI', 'C_FI > SE3', 'C_EE > FI', 'C_FI > EE', 'F_FI > SE1', 'F_SE1 > FI', 'F_FI > SE3', 'F_SE3 > FI', 'F_FI > EE', 'F_EE > FI', 'FI Buy', 'FI Sell'], axis=1)
X_temp2_index = X_temp2.index + DateOffset(hours=-hour_ahead)
X_temp2.set_index(X_temp2_index, inplace=True)
X_temp1 = X_temp1.to_frame()
X_temp = X_temp1.join(X_temp2)
X = X_temp.iloc[0:N_index[0] - hour_ahead]
X = X.to_numpy()
#X.reset_index(drop=True, inplace=True)
#X.columns = [''] * len(X.columns)
Pred_index = X.shape
print("Shape = "+str(Pred_index))


class DecisionTree():
    def __init__(self, x, y, n_features, f_idxs, idxs=None, depth=3, min_leaf=5):
        if idxs is None: idxs=np.arange(len(y))
        self.x, self.y, self.idxs, self.min_leaf, self.f_idxs = x, y, idxs, min_leaf, f_idxs
        self.depth = depth
        #print(f_idxs)
        #         print(self.depth)
        self.n_features = n_features
        self.n, self.c = len(idxs), x.shape[1]
        self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.find_varsplit()

    def find_varsplit(self):
        for i in self.f_idxs: self.find_better_split(i)
        if self.is_leaf: return
        x = self.split_col
        lhs = np.nonzero(x <= self.split)[0]
        rhs = np.nonzero(x > self.split)[0]
        lf_idxs = np.random.permutation(self.x.shape[1])[:self.n_features]
        rf_idxs = np.random.permutation(self.x.shape[1])[:self.n_features]
        self.lhs = DecisionTree(self.x, self.y, self.n_features, lf_idxs, self.idxs[lhs], depth=self.depth - 1,
                                min_leaf=self.min_leaf)
        self.rhs = DecisionTree(self.x, self.y, self.n_features, rf_idxs, self.idxs[rhs], depth=self.depth - 1,
                                min_leaf=self.min_leaf)

    def find_better_split(self, var_idx):
        x, y = self.x[self.idxs, var_idx], self.y[self.idxs]
        sort_idx = np.argsort(x)
        sort_y, sort_x = y[sort_idx], x[sort_idx]
        rhs_sum2: object
        rhs_cnt, rhs_sum, rhs_sum2 = self.n, sort_y.sum(), (sort_y ** 2).sum()
        lhs_cnt, lhs_sum, lhs_sum2 = 0, 0., 0.

        for i in range(0, self.n - self.min_leaf - 1):
            xi, yi = sort_x[i], sort_y[i]
            lhs_cnt += 1
            rhs_cnt -= 1
            lhs_sum += yi
            rhs_sum -= yi
            lhs_sum2 += yi ** 2
            rhs_sum2 -= yi ** 2
            if i < self.min_leaf or xi == sort_x[i + 1]:
                continue

            lhs_std = std_agg(lhs_cnt, lhs_sum, lhs_sum2)
            rhs_std = std_agg(rhs_cnt, rhs_sum, rhs_sum2)
            curr_score = lhs_std * lhs_cnt + rhs_std * rhs_cnt
            if curr_score < self.score:
                self.var_idx, self.score, self.split = var_idx, curr_score, xi

    @property
    def split_name(self):
        return self.x.columns[self.var_idx]

    @property
    def split_col(self):
        return self.x[self.idxs, self.var_idx]

    @property
    def is_leaf(self):
        return self.score == float('inf') or self.depth <= 0

    def predict(self, x):
        return np.array([self.predict_row(xi) for xi in x])

    def predict_row(self, xi):
        if self.is_leaf: return self.val
        t = self.lhs if xi[self.var_idx] <= self.split else self.rhs
        return t.predict_row(xi)

def std_agg(cnt, s1, s2):
    try:
        return math.sqrt((s2 / cnt) - (s1 / cnt) ** 2)
    except:
        return 0

def create_tree(x, y, n_features, sample_sz, depth=3, min_leaf=5):
    idxs = np.random.permutation(len(y))[:sample_sz]
    f_idxs = np.random.permutation(x.shape[1])[:n_features]
    return DecisionTree(x[idxs], y[idxs], n_features, f_idxs,
                        idxs=np.array(range(sample_sz)), depth=depth, min_leaf=min_leaf)




from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit()
mse = []
rmse = []
mae = []
print(tscv)
TimeSeriesSplit(n_splits=5)
count = 0
for train_index, test_index in tscv.split(X):
    print("Cross Validation Loop: " + str(count))
    print("hour_ahead = " + str(hour_ahead))
    print("n_estimators = " + str(n_estimators))
    print("learning_rate = " + str(learning_rate))
    print("Depth = " + str(Depth))
    print("Min_leaf = " + str(Min_leaf))

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    xi = X_train
    train_index = X_train.shape
    yi = y_train - np.mean(y_train)
    ei = 0  # initialization of error
    predf = np.mean(y_train)  # initial prediction 0

    trees = []
    for i in range(n_estimators):
        tree = create_tree(xi, yi, train_index[1], train_index[0], depth=Depth, min_leaf=Min_leaf)
        predi = tree.predict(xi)
        predf = predf + learning_rate * predi
        print(str(i)+str("  ")+str(mean_absolute_error(y_train, predf)))
        ei = y_train - predf  # needed originl y here as residual always from original y
        yi = ei  # update yi as residual to reloop
        trees.append(tree)


    MSE = mean_squared_error(y_train, predf, squared=True)
    print("Training MSE  " + str(MSE))
    RMSE = np.sqrt(MSE)
    print("Training RMSE  " + str(RMSE))
    MAE = mean_absolute_error(y_train, predf)
    print("Training MAE  " + str(MAE))



    xi = X_test
    train_index = X_test.shape
    yi = y_test - np.mean(y_test)
    ei = 0  # initialization of error
    predf = np.mean(y_test)

    i = 0
    for t in trees:
        predi = t.predict(X_test)
        predf = predf + learning_rate * predi
        print(str(i)+str("  ")+str(mean_absolute_error(y_test, predf)))
        ei = y_test - predf  # needed originl y here as residual always from original y
        yi = ei  # update yi as residual to reloop

        #if i == 1:
        #   break
        i += 1


    MSE = mean_squared_error(y_test, predf, squared=True)
    mse.append(MSE)
    print("Testing MSE  " + str(MSE))
    RMSE = np.sqrt(MSE)
    rmse.append(RMSE)
    print("Testing RMSE  " + str(RMSE))
    MAE = mean_absolute_error(y_test, predf)
    mae.append(MAE)
    print("Testing MAE  " + str(MAE))
    count += 1


print("Avarage Validate MSE: " + str(np.mean(mse)))
print("Avarage Validate RMSE: " + str(np.mean(rmse)))
print("Avarage Validate MAE: " + str(np.mean(mae)))

Shape = (10976, 30)
TimeSeriesSplit(max_train_size=None, n_splits=5)
Cross Validation Loop: 0
hour_ahead = 24
n_estimators = 98
learning_rate = 0.1
Depth = 10
Min_leaf = 256
0  4.443448985657924
1  4.232409345858784
2  4.063761599748875
3  3.9270417999133107
4  3.8103944748257663
5  3.698498154673716
6  3.6219811776081032
7  3.561698242135401
8  3.4987216596951964
9  3.447395866247035
10  3.407662537142622
11  3.372109824261987
12  3.3429078010764215
13  3.325044871565828
14  3.312364611055904
15  3.302845021821409
16  3.2867542906980813
17  3.2823571891643453
18  3.2785336643291356
19  3.277078761108733
20  3.2757773430714177
21  3.2762330854611714
22  3.276572380297038
23  3.2780158213659654
24  3.279944406809533
25  3.2809010764591213
26  3.283354823602213
27  3.2858762241427706
28  3.28652231166166
29  3.288270865110956
30  3.290013217228132
31  3.291744750368942
32  3.293383713761651
33  3.2948437734478926
34  3.2963045374708226
35  3.297085292624678
36  3.298388872163738
37  3.29

In [7]:
import pandas as pd
import numpy as np
from IPython.display import display
from fastai.imports import *
from sklearn import metrics
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

hour_ahead = 24
N_tree = 50
Depth = 10
Min_leaf = 150


path = "/content/drive/MyDrive/Colab Notebooks/Project/star_data_set.csv"
Merge_all = pd.read_csv(path)
dti = pd.to_datetime(Merge_all['DateTime'], format='%Y-%m-%d %H', exact=False)
Merge_all.set_index(pd.Index(dti), inplace=True)
Merge_all.drop(['DateTime'], axis=1, inplace=True)

First = Merge_all.iloc[0:11000, :]
N_index = First.shape
Y_temp = First['FI']
Y = Y_temp.iloc[hour_ahead:N_index[0]]
Y = Y.to_numpy()
#Y.reset_index(drop=True, inplace=True)
#Y.columns = [''] * len(Y.columns)


X_temp1 = First['FI']
X_temp2 = First.loc[:, First.columns != 'FI']
#X_temp1 = First.drop(First.columns.difference(['FI', 'C_SE1 > FI', 'C_FI > SE1', 'C_SE3 > FI', 'C_FI > SE3', 'C_EE > FI', 'C_FI > EE', 'F_FI > SE1', 'F_SE1 > FI', 'F_FI > SE3', 'F_SE3 > FI', 'F_FI > EE', 'F_EE > FI', 'FI Buy', 'FI Sell']), axis=1)
#X_temp2 = First.drop(['FI', 'C_SE1 > FI', 'C_FI > SE1', 'C_SE3 > FI', 'C_FI > SE3', 'C_EE > FI', 'C_FI > EE', 'F_FI > SE1', 'F_SE1 > FI', 'F_FI > SE3', 'F_SE3 > FI', 'F_FI > EE', 'F_EE > FI', 'FI Buy', 'FI Sell'], axis=1)
X_temp2_index = X_temp2.index + DateOffset(hours=-hour_ahead)
X_temp2.set_index(X_temp2_index, inplace=True)
X_temp1 = X_temp1.to_frame()
X_temp = X_temp1.join(X_temp2)
X = X_temp.iloc[0:N_index[0] - hour_ahead]
X = X.to_numpy()
#X.reset_index(drop=True, inplace=True)
#X.columns = [''] * len(X.columns)
Pred_index = X.shape
print("Shape = "+str(Pred_index))


class RandomForest():
    def __init__(self, x, y, n_trees, n_features, sample_sz, depth=10, min_leaf=10):
        np.random.seed(12)
        if n_features == 'sqrt':
            self.n_features = int(np.sqrt(x.shape[1]))
        elif n_features == 'log2':
            self.n_features = int(np.log2(x.shape[1]))
        else:
            self.n_features = n_features
        print(self.n_features, "sha: ", x.shape[1])
        self.x, self.y, self.sample_sz, self.depth, self.min_leaf = x, y, sample_sz, depth, min_leaf
        self.trees = [self.create_tree() for i in range(n_trees)]

    def create_tree(self):
        idxs = np.random.permutation(len(self.y))[:self.sample_sz]
        f_idxs = np.random.permutation(self.x.shape[1])[:self.n_features]
        return DecisionTree(self.x[idxs], self.y[idxs], self.n_features, f_idxs,
                            idxs=np.array(range(self.sample_sz)), depth=self.depth, min_leaf=self.min_leaf)

    def predict(self, x):
        return np.mean([t.predict(x) for t in self.trees], axis=0)


def std_agg(cnt, s1, s2):
    try:
        return math.sqrt((s2 / cnt) - (s1 / cnt) ** 2)
    except:
        return 0


class DecisionTree():
    def __init__(self, x, y, n_features, f_idxs, idxs, depth, min_leaf):
        self.x, self.y, self.idxs, self.min_leaf, self.f_idxs = x, y, idxs, min_leaf, f_idxs
        self.depth = depth
        print(f_idxs)
        #         print(self.depth)
        self.n_features = n_features
        self.n, self.c = len(idxs), x.shape[1]
        self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.find_varsplit()

    def find_varsplit(self):
        for i in self.f_idxs: self.find_better_split(i)
        if self.is_leaf: return
        x = self.split_col
        lhs = np.nonzero(x <= self.split)[0]
        rhs = np.nonzero(x > self.split)[0]
        lf_idxs = np.random.permutation(self.x.shape[1])[:self.n_features]
        rf_idxs = np.random.permutation(self.x.shape[1])[:self.n_features]
        self.lhs = DecisionTree(self.x, self.y, self.n_features, lf_idxs, self.idxs[lhs], depth=self.depth - 1,
                                min_leaf=self.min_leaf)
        self.rhs = DecisionTree(self.x, self.y, self.n_features, rf_idxs, self.idxs[rhs], depth=self.depth - 1,
                                min_leaf=self.min_leaf)

    def find_better_split(self, var_idx):
        x, y = self.x[self.idxs, var_idx], self.y[self.idxs]
        sort_idx = np.argsort(x)
        sort_y, sort_x = y[sort_idx], x[sort_idx]
        rhs_sum2: object
        rhs_cnt, rhs_sum, rhs_sum2 = self.n, sort_y.sum(), (sort_y ** 2).sum()
        lhs_cnt, lhs_sum, lhs_sum2 = 0, 0., 0.

        for i in range(0, self.n - self.min_leaf - 1):
            xi, yi = sort_x[i], sort_y[i]
            lhs_cnt += 1
            rhs_cnt -= 1
            lhs_sum += yi
            rhs_sum -= yi
            lhs_sum2 += yi ** 2
            rhs_sum2 -= yi ** 2
            if i < self.min_leaf or xi == sort_x[i + 1]:
                continue

            lhs_std = std_agg(lhs_cnt, lhs_sum, lhs_sum2)
            rhs_std = std_agg(rhs_cnt, rhs_sum, rhs_sum2)
            curr_score = lhs_std * lhs_cnt + rhs_std * rhs_cnt
            if curr_score < self.score:
                self.var_idx, self.score, self.split = var_idx, curr_score, xi

    @property
    def split_name(self):
        return self.x.columns[self.var_idx]

    @property
    def split_col(self):
        return self.x[self.idxs, self.var_idx]

    @property
    def is_leaf(self):
        return self.score == float('inf') or self.depth <= 0

    def predict(self, x):
        return np.array([self.predict_row(xi) for xi in x])

    def predict_row(self, xi):
        if self.is_leaf: return self.val
        t = self.lhs if xi[self.var_idx] <= self.split else self.rhs
        return t.predict_row(xi)



from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit()
mse = []
rmse = []
mae = []
print(tscv)
TimeSeriesSplit(n_splits=5)
count = 0
for train_index, test_index in tscv.split(X):
    print("Cross Validation Loop: " + str(count))
    print("hour_ahead = " + str(hour_ahead))
    print("N_tree = " + str(N_tree))
    print("Depth = " + str(Depth))
    print("Min_leaf = " + str(Min_leaf))


    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    train_index = X_train.shape
    forests = RandomForest(X_train, y_train, N_tree, train_index[1], train_index[0], depth=Depth, min_leaf=Min_leaf)
    Pred = forests.predict(X_train)

    print("hour_ahead = " + str(hour_ahead))
    print("N_tree = " + str(N_tree))
    print("Depth = " + str(Depth))
    print("Min_leaf = " + str(Min_leaf))

    MSE = mean_squared_error(y_train, Pred, squared=True)
    print("Training MSE  " + str(MSE))
    RMSE = np.sqrt(MSE)
    print("Training RMSE  " + str(RMSE))
    MAE = mean_absolute_error(y_train, Pred)
    print("Training MAE  " + str(MAE))


    T_Pred = forests.predict(X_test)
    MSE = mean_squared_error(y_test, T_Pred, squared=True)
    mse.append(MSE)
    print("Testing MSE  " + str(MSE))
    RMSE = np.sqrt(MSE)
    rmse.append(RMSE)
    print("Testing RMSE  " + str(RMSE))
    MAE = mean_absolute_error(y_test, T_Pred)
    mae.append(MAE)
    print("Testing MAE  " + str(MAE))


print("Avarage Validate MSE: " + str(np.mean(mse)))
print("Avarage Validate RMSE: " + str(np.mean(rmse)))
print("Avarage Validate MAE: " + str(np.mean(mae)))

เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย
 11  8 21 29 14 27]
[14  0 13 22  5  2 18  7  8 20  9 26 16 27  4 25 17  3 29 11 12 15 24  1
 21 19 10 28  6 23]
[21 27 23  9 18  7 28 13 16  2  5  6 22 20 10 17  8 29  0 11 25  3 24 12
 15 26  4 19 14  1]
[29 23 27  1  7  4 12 28 26  3 20 10 24 21  2 16 11 19  5 13 25  0  8 17
  9  6 22 15 14 18]
[14 23 11  4  8  7 13 24  1 21 15 28 10 19 16 20  0 17 18  3 22  6 27 29
 26 25 12  2  9  5]
[11 17 28  3 25 21  7  9  6 20 29  4 15 19  0  1 13 12 23 24 14 27 10  2
  8 22 18 16 26  5]
[23 11  9 16  2 18 24  4 13 12 10 19 22  8  6  1 25 27 21 14 20 17 26  3
  5 29  0 15 28  7]
[29 12 11 19 15  0 14 28 27 16  8 24 25 26  5  2 10  6  9 22  1 13 23 18
  4  3 17 21  7 20]
[ 0  4  5  1 25 14 12 23 10 27  8  9 29 26  6 18 11 21 22 15 20 24 13  3
  2  7 19 17 28 16]
[11  2 22 18  7 27 24 17  4 25  0 19 29 26  8  5  6 20 10  3 21  9 16 15
 12 14 23  1 13 28]
[25  5 26 27  1  2 28  7 19 10  6 11 12 20  0 21  8 14 18  3  9 13 17 15
 24 23 22 29 

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from fastai.imports import *
from sklearn import metrics
from pandas.tseries.offsets import DateOffset
import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

hour_ahead = 24


path = "/content/drive/MyDrive/Colab Notebooks/Project/star_data_set.csv"
Merge_all = pd.read_csv(path)
dti = pd.to_datetime(Merge_all['DateTime'], format='%Y-%m-%d %H', exact=False)
Merge_all.set_index(pd.Index(dti), inplace=True)
Merge_all.drop(['DateTime'], axis=1, inplace=True)

First = Merge_all.iloc[0:11000, :]
N_index = First.shape
Y_temp = First['FI']
Y = Y_temp.iloc[hour_ahead:N_index[0]]
Y = Y.to_numpy()
#Y.reset_index(drop=True, inplace=True)
#Y.columns = [''] * len(Y.columns)


X_temp1 = First['FI']
X_temp2 = First.loc[:, First.columns != 'FI']
#X_temp1 = First.drop(First.columns.difference(['FI', 'C_SE1 > FI', 'C_FI > SE1', 'C_SE3 > FI', 'C_FI > SE3', 'C_EE > FI', 'C_FI > EE', 'F_FI > SE1', 'F_SE1 > FI', 'F_FI > SE3', 'F_SE3 > FI', 'F_FI > EE', 'F_EE > FI', 'FI Buy', 'FI Sell']), axis=1)
#X_temp2 = First.drop(['FI', 'C_SE1 > FI', 'C_FI > SE1', 'C_SE3 > FI', 'C_FI > SE3', 'C_EE > FI', 'C_FI > EE', 'F_FI > SE1', 'F_SE1 > FI', 'F_FI > SE3', 'F_SE3 > FI', 'F_FI > EE', 'F_EE > FI', 'FI Buy', 'FI Sell'], axis=1)
X_temp2_index = X_temp2.index + DateOffset(hours=-hour_ahead)
X_temp2.set_index(X_temp2_index, inplace=True)
X_temp1 = X_temp1.to_frame()
X_temp = X_temp1.join(X_temp2)
X = X_temp.iloc[0:N_index[0] - hour_ahead]
X = X.to_numpy()
#X.reset_index(drop=True, inplace=True)
#X.columns = [''] * len(X.columns)
Pred_index = X.shape
print("Shape = "+str(Pred_index))


from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
tscv = TimeSeriesSplit()
mse = []
rmse = []
mae = []
print(tscv)
TimeSeriesSplit(n_splits=5)
count = 0
for train_index, test_index in tscv.split(X):
    print("Cross Validation Loop: " + str(count))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    reg = LinearRegression().fit(X_train,y_train)


    predf = reg.predict(X_train)

    MSE = mean_squared_error(y_train, predf, squared=True)
    print("Training MSE  " + str(MSE))
    RMSE = np.sqrt(MSE)
    print("Training RMSE  " + str(RMSE))
    MAE = mean_absolute_error(y_train, predf)
    print("Training MAE  " + str(MAE))



    predf = reg.predict(X_test)

    MSE = mean_squared_error(y_test, predf, squared=True)
    mse.append(MSE)
    print("Testing MSE  " + str(MSE))
    RMSE = np.sqrt(MSE)
    rmse.append(RMSE)
    print("Testing RMSE  " + str(RMSE))
    MAE = mean_absolute_error(y_test, predf)
    mae.append(MAE)
    print("Testing MAE  " + str(MAE))



print("Avarage Validate MSE: " + str(np.mean(mse)))
print("Avarage Validate RMSE: " + str(np.mean(rmse)))
print("Avarage Validate MAE: " + str(np.mean(mae)))

Shape = (10976, 30)
TimeSeriesSplit(max_train_size=None, n_splits=5)
Cross Validation Loop: 0
Training MSE  35.87841009123533
Training RMSE  5.989858937507237
Training MAE  3.058020860387789
Testing MSE  60.91677880087178
Testing RMSE  7.804920166207453
Testing MAE  5.470018036759168
Cross Validation Loop: 0
Training MSE  38.7004412237035
Training RMSE  6.220967868724569
Training MAE  3.622073296150628
Testing MSE  78.24878472598654
Testing RMSE  8.8458343148618
Testing MAE  5.346475132107339
Cross Validation Loop: 0
Training MSE  50.058107980767616
Training RMSE  7.075175473496584
Training MAE  4.175350343430133
Testing MSE  142.69075039479355
Testing RMSE  11.945323369201587
Testing MAE  6.415324330389335
Cross Validation Loop: 0
Training MSE  72.56166019334773
Training RMSE  8.518313224655907
Training MAE  4.785356902810506
Testing MSE  92.96651545709699
Testing RMSE  9.641914512019747
Testing MAE  7.29791114282456
Cross Validation Loop: 0
Training MSE  73.54211218264399
Training RM

In [3]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import Ridge
tscv = TimeSeriesSplit()
mse = []
rmse = []
mae = []
print(tscv)
TimeSeriesSplit(n_splits=5)
count = 0
for train_index, test_index in tscv.split(X):
    print("Cross Validation Loop: " + str(count))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    clf = Ridge(alpha=32206)
    clf.fit(X_train,y_train)


    predf = clf.predict(X_train)

    MSE = mean_squared_error(y_train, predf, squared=True)
    print("Training MSE  " + str(MSE))
    RMSE = np.sqrt(MSE)
    print("Training RMSE  " + str(RMSE))
    MAE = mean_absolute_error(y_train, predf)
    print("Training MAE  " + str(MAE))



    predf = clf.predict(X_test)

    MSE = mean_squared_error(y_test, predf, squared=True)
    mse.append(MSE)
    print("Testing MSE  " + str(MSE))
    RMSE = np.sqrt(MSE)
    rmse.append(RMSE)
    print("Testing RMSE  " + str(RMSE))
    MAE = mean_absolute_error(y_test, predf)
    mae.append(MAE)
    print("Testing MAE  " + str(MAE))


print("Avarage Validate MSE: " + str(np.mean(mse)))
print("Avarage Validate RMSE: " + str(np.mean(rmse)))
print("Avarage Validate MAE: " + str(np.mean(mae)))

TimeSeriesSplit(max_train_size=None, n_splits=5)
Cross Validation Loop: 0
Training MSE  41.54217548776914
Training RMSE  6.445321984801779
Training MAE  3.4807418835128563
Testing MSE  60.22542248290806
Testing RMSE  7.760504009592937
Testing MAE  5.272711577244643
Cross Validation Loop: 0
Training MSE  43.619855477850706
Training RMSE  6.604532949259221
Training MAE  3.9057409506082794
Testing MSE  89.37393223672734
Testing RMSE  9.453778727933468
Testing MAE  5.538257230025161
Cross Validation Loop: 0
Training MSE  57.216749153922954
Training RMSE  7.564175378316063
Training MAE  4.346076330761025
Testing MSE  155.87931062298762
Testing RMSE  12.485163620192873
Testing MAE  6.57781211801131
Cross Validation Loop: 0
Training MSE  80.41718933213697
Training RMSE  8.967563176924765
Training MAE  4.987431701292915
Testing MSE  69.7135496939904
Testing RMSE  8.349464036331339
Testing MAE  5.918809482342149
Cross Validation Loop: 0
Training MSE  77.62646132310682
Training RMSE  8.810588023

In [4]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn import linear_model
tscv = TimeSeriesSplit()
mse = []
rmse = []
mae = []
print(tscv)
TimeSeriesSplit(n_splits=5)
count = 0
for train_index, test_index in tscv.split(X):
    print("Cross Validation Loop: " + str(count))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    clf = linear_model.Lasso(alpha=1.9)
    clf.fit(X_train,y_train)

    predf = clf.predict(X_train)

    MSE = mean_squared_error(y_train, predf, squared=True)
    print("Training MSE  " + str(MSE))
    RMSE = np.sqrt(MSE)
    print("Training RMSE  " + str(RMSE))
    MAE = mean_absolute_error(y_train, predf)
    print("Training MAE  " + str(MAE))



    predf = clf.predict(X_test)

    MSE = mean_squared_error(y_test, predf, squared=True)
    mse.append(MSE)
    print("Testing MSE  " + str(MSE))
    RMSE = np.sqrt(MSE)
    rmse.append(RMSE)
    print("Testing RMSE  " + str(RMSE))
    MAE = mean_absolute_error(y_test, predf)
    mae.append(MAE)
    print("Testing MAE  " + str(MAE))



print("Avarage Validate MSE: " + str(np.mean(mse)))
print("Avarage Validate RMSE: " + str(np.mean(rmse)))
print("Avarage Validate MAE: " + str(np.mean(mae)))

TimeSeriesSplit(max_train_size=None, n_splits=5)
Cross Validation Loop: 0
Training MSE  41.32371158036367
Training RMSE  6.428352166796999
Training MAE  3.305143045777778
Testing MSE  52.49706497691775
Testing RMSE  7.245485834429445
Testing MAE  4.761213368633077
Cross Validation Loop: 0
Training MSE  44.050095436326146
Training RMSE  6.6370245921140105
Training MAE  3.8543592344104507
Testing MSE  87.42129593423859
Testing RMSE  9.349935611234903
Testing MAE  5.465893709755962
Cross Validation Loop: 0
Training MSE  58.352322476611114
Training RMSE  7.638869188342677
Training MAE  4.382605595427045
Testing MSE  155.05687368250491
Testing RMSE  12.452183490557184
Testing MAE  6.5777314264180795
Cross Validation Loop: 0
Training MSE  82.07053224494008
Training RMSE  9.059278792759393
Training MAE  5.035707540469714
Testing MSE  68.54655008463003
Testing RMSE  8.279284394476978
Testing MAE  5.712483059184748
Cross Validation Loop: 0
Training MSE  79.21898576629638
Training RMSE  8.900504

In [5]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import ElasticNet 
tscv = TimeSeriesSplit()
mse = []
rmse = []
mae = []
print(tscv)
TimeSeriesSplit(n_splits=5)
count = 0
for train_index, test_index in tscv.split(X):
    print("Cross Validation Loop: " + str(count))
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    regr = ElasticNet(alpha=1.9, l1_ratio=0.5)
    regr.fit(X_train,y_train)

    predf = regr.predict(X_train)

    MSE = mean_squared_error(y_train, predf, squared=True)
    print("Training MSE  " + str(MSE))
    RMSE = np.sqrt(MSE)
    print("Training RMSE  " + str(RMSE))
    MAE = mean_absolute_error(y_train, predf)
    print("Training MAE  " + str(MAE))



    predf = regr.predict(X_test)

    MSE = mean_squared_error(y_test, predf, squared=True)
    mse.append(MSE)
    print("Testing MSE  " + str(MSE))
    RMSE = np.sqrt(MSE)
    rmse.append(RMSE)
    print("Testing RMSE  " + str(RMSE))
    MAE = mean_absolute_error(y_test, predf)
    mae.append(MAE)
    print("Testing MAE  " + str(MAE))



print("Avarage Validate MSE: " + str(np.mean(mse)))
print("Avarage Validate RMSE: " + str(np.mean(rmse)))
print("Avarage Validate MAE: " + str(np.mean(mae)))

TimeSeriesSplit(max_train_size=None, n_splits=5)
Cross Validation Loop: 0
Training MSE  40.91384678362827
Training RMSE  6.396393263678233
Training MAE  3.3047522979585437
Testing MSE  53.931904753346714
Testing RMSE  7.343834472082463
Testing MAE  4.8345937875155975
Cross Validation Loop: 0
Training MSE  43.814427107842896
Training RMSE  6.619246717553509
Training MAE  3.8417705297157525
Testing MSE  88.14316603127423
Testing RMSE  9.388459193673594
Testing MAE  5.4468552542085655
Cross Validation Loop: 0
Training MSE  57.920839466365074
Training RMSE  7.610574187692087
Training MAE  4.349398091857322
Testing MSE  155.12221847008482
Testing RMSE  12.454807042667696
Testing MAE  6.559040261290695
Cross Validation Loop: 0
Training MSE  81.63553069383225
Training RMSE  9.035238275432047
Training MAE  5.0062061294339735
Testing MSE  68.68637964587617
Testing RMSE  8.287724636224118
Testing MAE  5.740758441940625
Cross Validation Loop: 0
Training MSE  78.81595219850644
Training RMSE  8.877